In [1]:
import pandas as pd
#import numpy as np
#import glob
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

# Load Data

In [2]:
#data_dir = '../input/lish-moa'
data_dir = '../data/01_raw'

sample_submission = pd.read_csv(data_dir+'/sample_submission.csv')
train_targets_scored = pd.read_csv(data_dir+'/train_targets_scored.csv')
targets_scored = train_targets_scored.columns[1:]

# Launch data pipelines in parallel

In [13]:
#launch models
import subprocess
import time
import random

lsub = []

# mdl1 = subprocess.Popen("python data_v1.py".split())
# lsub += [mdl1]
mdl1b = subprocess.Popen("python data_v3b.py".split())
lsub += [mdl1b]
# mdl2 = subprocess.Popen("python data_v2.py".split())
# lsub += [mdl2]
# mdl3 = subprocess.Popen("python data_v3.py".split())
# lsub += [mdl3]

while len(lsub) > 0:
    random.shuffle(lsub)
    it = iter(lsub)
    p = next(it)

    print(p.args)
    if p.poll() != None:
        print(f"{p.args} is done")
        lsub.remove(p)
    time.sleep(10)

<subprocess.Popen object at 0x7fc4000872d0> is done


# Launch models in parallel

In [3]:
#All models are based on 5folds - 3 seeds
#model1 - DNN on scored targets only
#model2 - DNN on scored targets and unscored targets
#model3 - Resnet using scored targets and 
#model4 - tabnet model

#launch models
import subprocess
import time
import random

lsub = []

mdl1 = subprocess.Popen("python mdl1.py".split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
lsub += [mdl1]
mdl2 = subprocess.Popen("python mdl2.py".split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
lsub += [mdl2]
mdl3 = subprocess.Popen("python mdl3.py".split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
lsub += [mdl3]




while len(lsub) > 0:
    random.shuffle(lsub)
    it = iter(lsub)
    p = next(it)

    # print(p.communicate()[0])
    print(p.args)
    if p.poll() != None:
        print(f"{p.args} is done")
        lsub.remove(p)
    time.sleep(10)
    

['python', 'mdl3.py']
['python', 'mdl3.py']
['python', 'mdl3.py']
['python', 'mdl1.py']
['python', 'mdl2.py']
['python', 'mdl2.py']
['python', 'mdl1.py']
['python', 'mdl3.py']
['python', 'mdl3.py']
['python', 'mdl3.py']
['python', 'mdl1.py']
['python', 'mdl3.py']
['python', 'mdl3.py']
['python', 'mdl3.py']
['python', 'mdl1.py']
['python', 'mdl1.py']
['python', 'mdl2.py']
['python', 'mdl2.py']
['python', 'mdl3.py']
['python', 'mdl2.py']
['python', 'mdl1.py']
['python', 'mdl1.py']
['python', 'mdl2.py']
['python', 'mdl2.py']
['python', 'mdl1.py']
['python', 'mdl1.py']
['python', 'mdl3.py']
['python', 'mdl1.py']
['python', 'mdl2.py']
['python', 'mdl1.py']
['python', 'mdl1.py']
['python', 'mdl2.py']
['python', 'mdl3.py']
['python', 'mdl3.py']
['python', 'mdl2.py']
['python', 'mdl1.py']
['python', 'mdl3.py']
['python', 'mdl1.py']
['python', 'mdl1.py']
['python', 'mdl2.py']
['python', 'mdl1.py']
['python', 'mdl2.py']
['python', 'mdl1.py']
['python', 'mdl3.py']
['python', 'mdl1.py']
['python',

# Optimize ratios

In [5]:
oof_v1 = pd.read_csv('./oof_v1.csv.gz',compression="gzip")
oof_v2 = pd.read_csv('./oof_v2.csv.gz',compression="gzip")
oof_v3 = pd.read_csv('./oof_v3.csv.gz',compression="gzip")

In [36]:
#Optimize ratios v1,v2...
ratios = [.35,.35,.3]
#ratios = [.0,.0,1]

train = oof_v2.copy()
train[targets_scored] =  ratios[0]*oof_v1[targets_scored] \
                         + ratios[1]*oof_v2[targets_scored] \
                         + ratios[2]*oof_v3[targets_scored]


In [37]:
#mdl1 datav2 - CV log_loss:  0.01612273916064027  - 0.6456645302982578
    #mdl1 datav3 - CV log_loss:  0.016076573782498754 - 0.6508584615567887
    #mdl1 datav3 - CV log_loss:  0.016066518969799066 - 0.6507665247472552
    #mdl1 datav3 [0.5, 0.35, 0.3, 0.25] - CV log_loss:  0.016064025163850172 - 0.6506508618872058
    #mdl1 datav3 [0.5, 0.35, 0.35, 0.3] - CV log_loss:  0.016064025163850172 - 0.6506508618872058
#mdl2 datav3 - CV log_loss:  0.016002962182328745 - 0.649883601325213
#mdl3 datav1 - CV log_loss:  0.016334877532401343 - 0.6409718409851376
    #mdl3 datav1 - pat_sch10,pat 40  - CV log_loss:  0.016323506996166224 - 0.6450274806973708
    #mdl3 datav1c - pat_sch10,pat 40  - CV log_loss:  0.016328237089782156 - 0.6514274670059218
    #mdl3 datav1c - pat_sch10,pat 40, n_d 24, n_a 64 - CV log_loss:  0.01627582951315354 - 0.6470267317729611


    #tabnet
    #datav1 - 24/64  CV log_loss:  0.016338515879056676 0.6432424662369459
    #datav1 - 16/64  CV log_loss:  0.016356804603262998 0.6472967466584303

    #datav1c 24/64 nshared 2 indp 2 - CV log_loss:  0.016295707697504228  0.6424207847007823
    #datav1c 24/64 nshared 1 indp 1 - CV log_loss:  0.016384444042879066  0.6517732857559961
    #datav1c 24/64 nshared 0 indp 2 - CV log_loss:  0.016334049200083876  0.6476813816964853
    #datav1c 24/64 nshared 1 indp 3 - CV log_loss:  0.016355521780600613  0.6443578692121475

In [38]:
y_true = train_targets_scored.iloc[:,1:].values
y_pred = train[targets_scored].values

score = 0
for i in range(len(targets_scored)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / len(targets_scored)
    
print("CV log_loss: ", score)
print(roc_auc_score(y_true,y_pred))

CV log_loss:  0.01580187072146891
0.6547396041996395


In [ ]:
#.35,.35,.3 - CV log_loss:  0.015831060922422473 - 0.6525849845540593
#.35,.35,.3 - CV log_loss:  0.01580056953208387 - 0.6549611343990086

In [16]:
sub_v1 = pd.read_csv('./sub_v1.csv.gz')
sub_v2 = pd.read_csv('./sub_v2.csv.gz')
sub_v3 = pd.read_csv('./sub_v3.csv.gz')

In [17]:
test = sub_v2.copy()
test[targets_scored] =  ratios[0]*sub_v2[targets_scored] \
                            + ratios[1]*sub_v3[targets_scored]

In [19]:
#Prepare submission
submission = pd.read_csv(data_dir+'/sample_submission.csv')

#Add blended ratios
submission.iloc[:, 1:] = test[targets_scored]
submission.to_csv('submission.csv', index=False)